#### funcom dataset preprocess

In [ ]:
import json
import os
import pickle

In [ ]:
DIR = 'funcom_processed'
OUTDIR = 'data'

In [ ]:
with open(os.path.join(DIR,'funcom_processed.pickle'),'rb') as f:
    all_data = pickle.load(f)

In [ ]:
# comments_l = list(comments.values())
# functions_l = list(functions.values())
functions_l  = list(all_data['Methods'].values)
comments_l  = list(all_data['Docstring'].values)

tr_end = 400000
val_st = tr_end
val_end = val_st+50000
tst_st = val_end
tst_end = tst_st+50000
comm_train = comments_l[:tr_end]
func_train = functions_l[:tr_end]
comm_val = comments_l[val_st:val_end]
func_val = functions_l[val_st:val_end]
comm_test = comments_l[tst_st:tst_end]
func_test = functions_l[tst_st:tst_end]

In [ ]:
def write_to_file(filepath, func_dict, comm_dict):
    with open(filepath,'w') as out:
        i=0
        for c,n in zip(func_dict,comm_dict):
            exam = {}
            exam['code']=c
            exam['nl']=n
            json.dump(exam, out)
            out.write("\n")
            i=i+1
            if i%10000==0:
                print("{} processed".format(i))

In [ ]:
comm_temp = comm_train[:10]
func_temp = func_train[:10]
write_to_file(os.path.join(OUTDIR,'temp.json'), func_temp, comm_temp)

In [ ]:
OUTDIR = 'data'
write_to_file(os.path.join(OUTDIR,'train.json'), func_train, comm_train)
write_to_file(os.path.join(OUTDIR,'valid.json'), func_val, comm_val)
write_to_file(os.path.join(OUTDIR,'test.json'), func_test, comm_test)

#### test change in preproc/java_fiel_creator.py

In [ ]:
import re
def extract_replacements(code):
    varEncDict = {}
    varDecDict = {}
#     print(code)
    codecopy = re.sub(r"\([A-z]+<.*>", "(type ", code)
    codecopy = re.sub(r"\,[A-z]+<.*>", ",type ", codecopy)
    codecopy = re.sub(r"@+.*(Test|Override)\s", "ftype", codecopy)
    dectype = ['public','private','static','protected']
    for dec in dectype:
        fresult = codecopy.find(dec)
        if fresult!=-1:
            codecopy = codecopy[fresult:]
            break
    codecopy = re.sub(r"@+.*(public|private|static|protected)\s", "declaration", codecopy)
    codecopy = codecopy.lstrip()
    decl = re.split('\n|{',codecopy)[0]
    print(decl)
    varDecl = re.findall("\((.*?)\)", decl)[0]
    varList = varDecl.split(",")

In [ ]:
code = '    @Override\n\tpublic void mousePressed(MouseEvent e) { // mac\n\tmaybeShowPopup(e);\n    }\n'
extract_replacements(code)

#### dataset buld commands

In [ ]:
!python3 preproc/java_files_creator.py

In [ ]:
!bash preproc/preprocess.sh

#### study result step 1

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel("results/Study result.xlsx", sheet_name='Copy').fillna(0)
cat = pd.read_excel("results/Study result.xlsx", sheet_name='catagories').fillna(0)
data=data.iloc[:150,:]
data = data.drop(columns=['Raw_Methods', 'Ground_Truth'])

##### column element count

In [ ]:
cat.head(1)

In [ ]:
data.columns

In [ ]:
all_count={}
all_count_df=pd.DataFrame()
for c in data.columns[6:]:
    all_count[c]=  dict.fromkeys(list(cat['original']),None)
    for d, coun in dict(data[c].value_counts()).items():
        all_count[c][d]=coun
all_count_df = pd.DataFrame(all_count).fillna(0)

In [ ]:
all_count_df = all_count_df.loc[~(all_count_df==0).all(axis=1)]

In [ ]:
all_count_df.columns

In [ ]:
cat_d = (dict(zip(cat['original'],cat['name'])))
all_count_df['index']=all_count_df.index
all_count_df['name']=all_count_df.apply(lambda x: cat_d[x['index']], axis=1)

import matplotlib.pyplot as plt

fig=plt.figure(figsize=(13,10))
fig.show()
ax=fig.add_subplot(211)
# ax.set_ylim([0, 5])
all_count_df.plot.bar(ax=ax, y= ['CodeBERT Conflict', 'NeuralCodeSum Conflict', 
                                'Code2Seq Conflict'],x='name',rot=90, width = 0.5)
plt.legend(["CodeBert", "NeuralCodeSum","Code2Seq"])
plt.legend(loc=2)
# for i in [0,3,13,15,19,26,31,36]:
#     ax.get_xticklabels()[i].set_color("red")
# ax.get_xticklabels()[13].set_color("blue")
plt.ylabel('Count')
plt.xlabel('Catagory Id')
plt.title('Error catagorical count')
plt.tight_layout()
plt.savefig('results/count_fig.pdf')
plt.draw()

In [ ]:
all_count_df.sum(axis = 0, skipna = True)

In [ ]:
all_count_df
all_count_df.to_csv('results/count_150.csv')

##### conflict calculation

In [ ]:
def conf_res(x,y,z):
#     if x!=0 or y!=0 or z!=0:
#         if x==y or x==z or y==z:
#             return 'no conflict'
#         else:
#             return 'conflict'
#     else:
#         return 'no pair'
    if x==0:
        if y!=0 and z!=0 and y!=z:
            return 'conflict'
        elif y!=0 and z!=0 and y==z:
            return 'no conflict'
        else:
            return 'no pair'
    if y==0:
        if x!=0 and z!=0 and x!=z:
            return 'conflict'
        elif x!=0 and z!=0 and x==z:
            return 'no conflict'
        else:
            return 'no pair'
    if z==0:
        if y!=0 and x!=0 and y!=x:
            return 'conflict'
        elif y!=0 and x!=0 and y==x:
            return 'no conflict'
        else:
            return 'no pair'
                
        
if __name__ == '__main__':
    cdata = data.copy()
    print(data.columns)
    cols = list(data.columns[6:])
    print(cols)
    cdata['codebert-u'] = cdata.apply(lambda x: conf_res(x[cols[0]],x[cols[3]], x[cols[6]]),axis=1)
    cdata['neural-u'] = cdata.apply(lambda x: conf_res(x[cols[1]],x[cols[4]], x[cols[7]]),axis=1)
    cdata['code2seq-u'] = cdata.apply(lambda x: conf_res(x[cols[2]],x[cols[5]], x[cols[8]]),axis=1)
    print("----",cdata.columns[18:])
    conflict_count={}
    for c in cdata.columns[18:]:
        conflict_count[c]= cat_count_dict = dict.fromkeys(list(cdata['codebert-u']),None)
        for d, coun in dict(cdata[c].value_counts()).items():
            conflict_count[c][d]=coun
    all_conflict_df = pd.DataFrame(conflict_count)    

In [ ]:
for i,c in enumerate(cdata.columns[8:]):
    print(i,c)

In [ ]:
cdata.columns[18:]

In [ ]:
all_conflict_df.to_csv('results/conflict_150.csv')

In [ ]:
all_conflict_df

### stat significance

##### preprocessing for code2seq codeber index and reference matching

In [ ]:
from collections import Counter
  
def removeElements(lst, k):
    counted = Counter(lst)
      
    temp_lst = []
    for el in counted:
        if counted[el] > k:
            temp_lst.append(el)
              
    res_lst = []
    for el in lst:
        if el not in temp_lst:
            res_lst.append(el)
              
    return(res_lst)
      
# Driver code


with open('stat/nc_ref.txt', 'r') as f:
    nc_ref = f.readlines()
    
with open('stat/cb_ref.txt', 'r') as f:
    cb_ref = f.readlines()

k = 1
cb_ref = [k.split('\t')[1][:-1] for k in cb_ref]
# cb_ref = [k.replace(',','') for k in cb_ref]
# cb_ref = [k.replace('\'','') for k in cb_ref]
# cb_ref = [k.replace('\"','') for k in cb_ref]
# cb_ref = [k.replace('\`','') for k in cb_ref]
# cb_ref = [k.replace('(','') for k in cb_ref]
# cb_ref = [k.replace(')','') for k in cb_ref]
nc_ref = [k.split('\t')[1][:-2] for k in nc_ref]
cb_to_k = {s:i for i,s in enumerate(cb_ref)}
nc_to_k = {s:i for i,s in enumerate(nc_ref)}
cb_ref_once = removeElements(cb_ref, k)
id_to_delete = set(cb_to_k.keys()).difference(set(cb_ref_once))
cb_to_k = {key: cb_to_k[key] for key in cb_to_k if key not in id_to_delete}
nc_to_k = {key: nc_to_k[key] for key in nc_to_k if key not in id_to_delete}
print(len(cb_to_k), len(nc_to_k))

In [ ]:
with open('stat/nc_pred.txt', 'r') as f:
    nc_pred = f.readlines()
nc_pred = [k.split('\t')[1][:-2] for k in nc_pred]

with open('stat/cb_pred.txt', 'r') as f:
    cb_pred = f.readlines()
cb_pred = [k.split('\t')[1][:-1] for k in cb_pred]

In [ ]:
all_info=[]
for i,d in enumerate(cb_ref):
    if (d in cb_to_k) and (d in nc_to_k):
        all_info.append([d, cb_pred[cb_to_k[d]], nc_pred[nc_to_k[d]]])
    else:
        all_info.append([(d,i), '#not_found#\n','#not_found#\n'])

cols  = ['ref','cb_pred','nc_pred']
all_data_pd = pd.DataFrame.from_records(all_info, columns=cols)

In [ ]:
with open('stat/cs_ref.txt', 'r') as f:
    nc_ref = f.readlines()
    
with open('stat/cb_ref.txt', 'r') as f:
    cb_ref = f.readlines()

k = 1
cb_ref = [k.split('\t')[1][:-1] for k in cb_ref]
cb_ref = [k.replace(',','') for k in cb_ref]
cb_ref = [k.replace('\'','') for k in cb_ref]
cb_ref = [k.replace('\"','') for k in cb_ref]
cb_ref = [k.replace('\`','') for k in cb_ref]
cb_ref = [k.replace('(','') for k in cb_ref]
cb_ref = [k.replace(')','') for k in cb_ref]
nc_ref = [k[:-1] for k in nc_ref]
cb_to_k = {s:i for i,s in enumerate(cb_ref)}
nc_to_k = {s:i for i,s in enumerate(nc_ref)}
cb_ref_once = removeElements(cb_ref, k)
id_to_delete = set(cb_to_k.keys()).difference(set(cb_ref_once))
cb_to_k = {key: cb_to_k[key] for key in cb_to_k if key not in id_to_delete}
nc_to_k = {key: nc_to_k[key] for key in nc_to_k if key not in id_to_delete}
print(len(cb_to_k), len(nc_to_k))

In [ ]:
with open('stat/cs_pred.txt', 'r') as f:
    nc_pred = f.readlines()
nc_pred = [k[:-1] for k in nc_pred]

with open('stat/cb_pred.txt', 'r') as f:
    cb_pred = f.readlines()
cb_pred = [k.split('\t')[1][:-1] for k in cb_pred]

In [ ]:
cs_info=[]
for i,d in enumerate(cb_ref):
    if (d in cb_to_k) and (d in nc_to_k):
        cs_info.append( nc_pred[nc_to_k[d]])
    else:
        cs_info.append('#not_found#\n')

In [ ]:
all_data_pd['cs_pred'] = cs_info
data = all_data_pd.copy()
data = data.loc[data['nc_pred']!='#not_found#\n']
data = data.loc[data['cs_pred']!='#not_found#\n']

In [ ]:
data

##### saving in different file to do bootstrap resampling significance test

In [ ]:
with open('stat/stat_ref.txt', 'w') as f:
    f.writelines(l+'\n' for l in list(data['ref']))
with open('stat/stat_cb.txt', 'w') as f:
    f.writelines(str(l)+'\n' for l in list(data['cb_pred']))
with open('stat/stat_nc.txt', 'w') as f:
    f.writelines(l+'\n' for l in list(data['nc_pred']))
with open('stat/stat_cs.txt', 'w') as f:
    f.writelines(l+'\n' for l in list(data['cs_pred']))

In [ ]:
!python3 paired-bootstrap.py stat/stat_ref.txt \
stat/stat_cb.txt \
stat/stat_nc.txt \
--eval_type bleu \
--num_samples 100

In [ ]:
!python3 paired-bootstrap.py stat/stat_ref.txt \
stat/stat_cb.txt \
stat/stat_nc.txt \
--eval_type bleu \
--num_samples 300

In [ ]:
!python3 paired-bootstrap.py stat/stat_ref.txt \
stat/stat_cb.txt \
stat/stat_cs.txt \
--eval_type bleu \
--num_samples 100